# Project 3: Binary classification for brest cancer with tuning

# Step 1

Libraries

In [4]:
!pip install skorch

In [5]:
import pandas as pd
import numpy as np
import torch.nn as nn
from skorch import NeuralNetBinaryClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV

# Step 2:

Database

In [6]:
forecasters = pd.read_csv('/content/entradas_breast.csv')
labels = pd.read_csv('/content/saidas_breast.csv')

In [7]:
np.random.seed(123)
torch.manual_seed(123)

In [8]:
forecasters = np.array(forecasters, dtype='float32')
labels = np.array(labels,dtype='float32' ).squeeze(1)

In [9]:
forecasters.shape

(569, 30)

In [10]:
labels.shape

(569,)

# Step 3

Class for neural network structure

In [11]:
class torch_classifier(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()

    # 30 -> 16 -> 16 -> 1

    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation

    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation


    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)

    # self.output = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)

    X = self.dense1(X)
    X = self.activation1(X)

    X = self.dense2(X)
    # X = self.output(X)

    return X

# Step 4

Skorch

In [12]:
sklearn_classifier = NeuralNetBinaryClassifier(module=torch_classifier,
                                               lr = 0.001,
                                               optimizer__weight_decay=0.0001,
                                               train_split=False)


# Step 5

Parameter tuning

In [13]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCEWithLogitsLoss], #, torch.nn.HingeEmbeddingLoss], # ** ATUALIZAÇÃO **
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16],
          'module__initializer': [torch.nn.init.uniform_]} # _, torch.nn.init.normal_]}

In [14]:
params

{'batch_size': [10],
 'max_epochs': [100],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD],
 'criterion': [torch.nn.modules.loss.BCEWithLogitsLoss],
 'module__activation': [<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  <function torch.nn.functional.tanh(input)>],
 'module__neurons': [8, 16],
 'module__initializer': [<function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>]}

In [15]:
grid_search = GridSearchCV(estimator=sklearn_classifier, param_grid=params, scoring='accuracy', cv=2)

grid_search.fit(forecasters, labels)

  epoch    train_loss     dur
-------  ------------  ------
      1    27299.8736  0.2268
      2    24343.2034  0.0588
      3    21607.9790  0.0784
      4    19135.1196  0.0887
      5    16931.4971  0.0839
      6    14986.2345  0.0746
      7    13263.7452  0.0706
      8    11731.9724  0.0672
      9    10367.9490  0.0651
     10     9150.4826  0.0657
     11     8060.7256  0.0677
     12     7080.4180  0.0962
     13     6192.2901  0.2918
     14     5382.8819  0.1431
     15     4640.1386  0.1951
     16     3952.5591  0.1762
     17     3309.9036  0.1888
     18     2702.8657  0.2401
     19     2122.0785  0.1009
     20     1558.6171  0.1324
     21     1003.4027  0.2108
     22      478.4378  0.1163
     23      138.4220  0.1278
     24       90.7287  0.1900
     25       88.3918  0.1745
     26       77.7050  0.0876
     27       70.4366  0.1244
     28       64.3088  0.0923
     29       58.6979  0.0758
     30       54.5392  0.0974
     31       52.0841  0.0927
     32   

GridSearchCV(cv=2,
             estimator=<class 'skorch.classifier.NeuralNetBinaryClassifier'>[uninitialized](
  module=<class '__main__.torch_classifier'>,
),
             param_grid={'batch_size': [10],
                         'criterion': [<class 'torch.nn.modules.loss.BCEWithLogitsLoss'>],
                         'max_epochs': [100],
                         'module__activation': [<function relu at 0x79c088f37490>,
                                                <function tanh at 0x79c088f37eb0>],
                         'module__initializer': [<function uniform_ at 0x79c088fab1c0>],
                         'module__neurons': [8, 16],
                         'optimizer': [<class 'torch.optim.adam.Adam'>,
                                       <class 'torch.optim.sgd.SGD'>]},
             scoring='accuracy')

In [16]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [17]:
best_parameters

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCEWithLogitsLoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
 'module__initializer': <function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [18]:
best_accuracy

0.7944897454904868